## COVID-19 detection using X-ray images usind Convolutional Neural Networks

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
import keras
from keras.layers import *
from keras.models import *
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.preprocessing import image
from tensorflow.keras.applications import VGG16, ResNet50
import matplotlib.pyplot as plt
import tensorflow as tf

COLLECTION OF DATA OF X-RAYS FROM MULTIPLE SOURCES

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3687, done.
remote: Total 3687 (delta 0), reused 0 (delta 0), pack-reused 3687
Receiving objects: 100% (3687/3687), 715.78 MiB | 11.16 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
Checking out files: 100% (1152/1152), done.


In [ ]:
import json

In [ ]:
with open('kaggle.json', 'w') as fp:
  json.dump(json.loads(b'{"username":"priyairagamreddy","key":"39325759e7597e230f3dd4ea7d5ddbee"}'), fp)

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"priyairagamreddy","key":"39325759e7597e230f3dd4ea7d5ddbee"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 2.28G/2.29G [00:50<00:00, 40.1MB/s]
100% 2.29G/2.29G [00:50<00:00, 48.7MB/s]


In [ ]:
file_path="/content/covid-chestxray-dataset/metadata.csv"
image_path="/content/covid-chestxray-dataset/images"

In [ ]:
df_xray=pd.read_csv(file_path)
df_xray.shape

(930, 29)

DATA OF POSITIVE COVID PATIENTS

In [ ]:
df_xray.head()

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,in_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
1,2,3.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
2,2,5.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
3,2,6.0,M,65.0,COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
4,4,0.0,F,52.0,COVID-19,Y,NaN,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN


In [ ]:
df=df_xray[(df_xray["finding"]=="COVID-19")&(df_xray["view"]=="PA")]
df.shape

(180, 29)

In [ ]:
! unzip -q /content/chest-xray-pneumonia.zip

In [ ]:
PREPARING THE DATA SET

In [ ]:
!mkdir Dataset
!mkdir Dataset/covid
!mkdir Dataset/normal

In [ ]:
for (i,row) in df.iterrows():
  filename = row["filename"]
  image_path_1 = os.path.join(image_path,filename)
  target_path_1 = os.path.join("/content/Dataset/covid",filename)
  shutil.copy2(image_path_1,target_path_1)

In [ ]:
import random
random.seed(15)
normal_sample_images=random.sample(os.listdir("/content/chest_xray/train/NORMAL"),180)

In [ ]:
for i in normal_sample_images:
  image_path_1=os.path.join("/content/chest_xray/train/NORMAL",i)
  target_path_1=os.path.join("/content/Dataset/normal",i)
  shutil.copy2(image_path_1,target_path_1)


In [ ]:
!rm -r /content/Dataset/.ipynb_checkpoints/

rm: cannot remove '/content/Dataset/.ipynb_checkpoints/': No such file or directory


In [ ]:
!mkdir dataset_final
!mkdir dataset_final/train
!mkdir dataset_final/validation
!mkdir dataset_final/train/covid
!mkdir dataset_final/validation/covid
!mkdir dataset_final/train/normal
!mkdir dataset_final/validation/normal

In [ ]:
covid_files = os.listdir('Dataset/covid')
normal_files = os.listdir('Dataset/normal')
print(len(covid_files), len(normal_files))
covid_files[0]

180 180


'gr1_lrg-b.jpg'

In [ ]:
random.shuffle(covid_files)
random.shuffle(normal_files)
covid_files[0]

'f6575117.jpg'

In [ ]:
for file in covid_files[:int(len(covid_files)*0.8)]:
  file_path = os.path.join('Dataset/covid', file)
  destination_path = os.path.join('dataset_final/train/covid', file)
  shutil.copy2(file_path, destination_path)


In [ ]:
for file in covid_files[int(len(covid_files)*0.8):]:
  file_path = os.path.join('Dataset/covid', file)
  destination_path = os.path.join('dataset_final/validation/covid', file)
  shutil.copy2(file_path, destination_path)

In [ ]:
for file in normal_files[:int(len(normal_files)*0.8)]:
  file_path = os.path.join('Dataset/normal', file)
  destination_path = os.path.join('dataset_final/train/normal', file)
  shutil.copy2(file_path, destination_path)

In [ ]:
for file in normal_files[int(len(normal_files)*0.8):]:
  file_path = os.path.join('Dataset/normal', file)
  destination_path = os.path.join('dataset_final/validation/normal', file)
  shutil.copy2(file_path, destination_path)

## Following this guide:
## https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
from keras import optimizers

IMPLEMENTING THE MODEL USING **DATA AUGMENTATION** AND **DROPOUT REGULARIZATION**

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data

train_generator = train_datagen.flow_from_directory(
        'dataset_final/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'dataset_final/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')


Found 288 images belonging to 2 classes.
Found 72 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
Model=Sequential()
Model.add(Conv2D(32,kernel_size=(3,3),activation="relu",input_shape=(224,224,3)))
Model.add(Conv2D(64,(3,3),activation="relu"))
Model.add(MaxPooling2D(pool_size=(2,2)))
Model.add(Dropout(0.25))

Model.add(Conv2D(64,(3,3),activation="relu"))
Model.add(MaxPooling2D(pool_size=(2,2)))
Model.add(Dropout(0.25))

Model.add(Conv2D(128,(3,3),activation="relu"))
Model.add(MaxPooling2D(pool_size=(2,2)))
Model.add(Dropout(0.25))

Model.add(Flatten())
Model.add(Dense(64,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(1,activation="sigmoid"))

Model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=["accuracy"])



In [ ]:
Model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 54, 54, 64)       

In [ ]:
callback=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [ ]:
hist=Model.fit_generator(train_generator,steps_per_epoch=8,epochs=10,
        validation_data=validation_generator,
        validation_steps=2,callbacks=[callback])

Epoch 1/10
8/8 [==============================] - 36s 5s/step - loss: 0.3098 - accuracy: 0.8672 - val_loss: 0.1984 - val_accuracy: 0.9062
Epoch 2/10
8/8 [==============================] - 36s 5s/step - loss: 0.2482 - accuracy: 0.9141 - val_loss: 0.0725 - val_accuracy: 1.0000
Epoch 3/10
8/8 [==============================] - 36s 4s/step - loss: 0.2402 - accuracy: 0.8906 - val_loss: 0.1215 - val_accuracy: 1.0000
Epoch 4/10
8/8 [==============================] - 35s 4s/step - loss: 0.2122 - accuracy: 0.9219 - val_loss: 0.0338 - val_accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 36s 4s/step - loss: 0.2482 - accuracy: 0.8984 - val_loss: 0.0586 - val_accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 35s 4s/step - loss: 0.2874 - accuracy: 0.8750 - val_loss: 0.1645 - val_accuracy: 0.9688


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

SAVE THE MODEL AND EVALUATE

In [ ]:
Model.save("mod_covid.h5")

In [ ]:
Model.evaluate_generator(train_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.26503944396972656, 0.90625]

In [ ]:
Model.evaluate_generator(validation_generator)

[0.16712604463100433, 0.9583333134651184]

In [ ]:
train_generator.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
Model=load_model("mod_covid.h5")

In [ ]:
y_actual =[]
y_test = []

for i in os.listdir("/content/dataset_final/validation/normal"):
    img = image.load_img("/content/dataset_final/validation/normal/"+i, target_size=(224,224))
    img =image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    p =Model.predict_classes(img)
    y_test.append(p[0,0])
    y_actual.append(1)

for i in os.listdir("/content/dataset_final/validation/covid"):
    img = image.load_img("/content/dataset_final/validation/covid/"+i, target_size=(224,224))
    img =image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    p =Model.predict_classes(img)
    y_test.append(p[0,0])
    y_actual.append(0)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
report=classification_report(y_actual, y_test,target_names=['covid','normal'])
print(report)

              precision    recall  f1-score   support

       covid       0.86      1.00      0.92        36
      normal       1.00      0.83      0.91        36

    accuracy                           0.92        72
   macro avg       0.93      0.92      0.92        72
weighted avg       0.93      0.92      0.92        72



In [ ]:
ACHEIVED THE ACCURACY AND RECALL OF 92%